In [2]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
import tempfile
import os

# Búsqueda BLASTp con una secuencia de proteína

En esta práctica se desarrolla un programa en Python que realiza una búsqueda de similitud entre una secuencia de proteína introducida por el usuario y una base de datos de proteínas. Para ello se utiliza la herramienta BLASTp (Basic Local Alignment Search Tool for Proteins), que permite identificar qué secuencias conocidas presentan mayor homología con la secuencia consultada.

El ejercicio se divide en dos partes claramente diferenciadas:
(1) una búsqueda online, enviada directamente a los servidores del NCBI, y
(2) una búsqueda local, realizada sobre una base de datos almacenada en nuestro propio ordenador.

A continuación se describe detalladamente el procedimiento seguido para garantizar un análisis robusto, reproducible y científicamente válido.

## 1. Introducción a BLASTp

BLASTp compara una secuencia de aminoácidos con una base de datos de proteínas para identificar:
- Secuencias similares evolutivamente.
- Posibles funciones asociadas.
- Proteínas homólogas en otras especies.

El criterio más utilizado para evaluar la relevancia biológica de un alineamiento es el E-value, que indica la probabilidad de obtener un alineamiento igual o mejor por azar. Un E-value muy bajo (por ejemplo < 0.001) indica un resultado altamente significativo.

## 2. Entrada de datos: validación y limpieza de la secuencia

El usuario introduce una secuencia de aminoácidos. Para evitar errores durante el BLAST, se implementa una función de validación que:

1. Elimina espacios en blanco o saltos de línea.
2. Convierte la secuencia a mayúsculas.
3. Comprueba que solo contiene aminoácidos válidos (20 aminoácidos estándar + X).

Si se detecta cualquier carácter no permitido, el programa devuelve un mensaje de error explicativo.

Este control garantiza que la consulta enviada al servidor o al programa local sea correcta y evita interrupciones en la ejecución.

In [3]:
def limpiar_proteina(seq: str) -> str:
    seq = seq.replace(" ", "").replace("\n", "").upper()
    validos = set("ACDEFGHIKLMNPQRSTVWYX")

    if not seq:
        raise ValueError("La secuencia está vacía.")

    if not set(seq).issubset(validos):
        raise ValueError("La secuencia contiene símbolos no válidos para proteína.")

    return seq


def extraer_hits_filtrados(record, threshold=0.001):
    resultados = []

    for aln in record.alignments:
        hsp = aln.hsps[0]  
        if hsp.expect < threshold:
            identidad = (hsp.identities / hsp.align_length) * 100

            resultados.append({
                "id": aln.hit_id,
                "len": aln.length,
                "evalue": hsp.expect,
                "identidad": identidad
            })

    return resultados


def guardar_resultados(rutasalida, resultados):
    with open(rutasalida, "w") as f:
        f.write("ID\tLongitud\tE-value\t% Identidad\n")
        for r in resultados:
            f.write(f"{r['id']}\t{r['len']}\t{r['evalue']:.3e}\t{r['identidad']:.2f}\n")

## 3. Búsqueda BLASTp online

Para realizar la búsqueda online se utiliza la función NCBIWWW.qblast de Biopython, indicando:
- `program="blastp"`
- `database="nr"` (base de datos no redundante de proteínas)
- La secuencia validada por el usuario

El servidor devuelve una respuesta en formato XML, que se procesa con `NCBIXML.read`-

Una vez parseado el resultado, se recorren todos los alineamientos y se seleccionan únicamente aquellos cuya mejor región alineada (HSP) presenta E-value < 0.001.

Para cada alineamiento filtrado se extraen:

- Identificador del hit
- Longitud de la proteína anotada
- E-value

Porcentaje de identidad, calculado  el número de posiciones idénticas entre la longitud del alineamiento por 100.

Todos estos resultados se guardan en un fichero de salida (resultados_online_proteina.txt), convenientemente formateado en columnas para facilitar su consulta.

In [5]:
seq_usuario = input("Introduce una secuencia de PROTEÍNA: ")

try:
    seq_limpia = limpiar_proteina(seq_usuario)
except ValueError as e:
    print(f"Error: {e}")
    raise SystemExit

print("\nRealizando BLASTp online... (esto puede tardar un poco)")

handle = NCBIWWW.qblast(
    program="blastp",
    database="nr",
    sequence=seq_limpia
)

record = NCBIXML.read(handle)

resultados_online = extraer_hits_filtrados(record)

guardar_resultados("resultados_online_proteina.txt", resultados_online)

print("\n--- RESULTADOS ONLINE ---")
print(f"Total hits con E-value < 0.001: {len(resultados_online)}")
print("Guardado en 'resultados_online_proteina.txt'")



Realizando BLASTp online... (esto puede tardar un poco)

--- RESULTADOS ONLINE ---
Total hits con E-value < 0.001: 0
Guardado en 'resultados_online_proteina.txt'


## 4. Construcción de la base de datos BLAST local

Para realizar la búsqueda local, primero es necesario disponer de un conjunto de proteínas en formato FASTA (.faa, .fa o .fasta).
Este archivo puede obtenerse desde UniProt, Ensembl u otras bases de datos especializadas.

A partir de este fichero, se construye una base de datos BLAST local mediante el comando: `makeblastdb -in proteinas.faa -dbtype prot -out mi_db_prot`

Esto genera los archivos índice necesarios (.pin, .phr, .psq) para que BLASTp pueda realizar la búsqueda local.

## 5. Búsqueda BLASTp local

Una vez creada la base de datos, el script ejecuta BLASTp de forma local mediante la clase `NcbiblastpCommandline`, que permite llamar al ejecutable `blastp` desde Python.

El proceso consiste en:
1. Crear un archivo FASTA temporal con la secuencia introducida por el usuario.
2. Ejecutar el comando BLASTp indicando:
- La ruta del ejecutable (`blastp`)
- La base de datos local (`mi_db_prot`)
- El formato XML como salida (`outfmt=5`)
3. Leer el fichero XML generado
4. Aplicar el mismo criterio de filtrado empleado en la búsqueda online (E-value < 0.001).
5. Guardar los resultados en resultados_local_proteina.txt.

De esta forma, se obtiene un análisis reproducible sin depender de los servidores externos del NCBI.


In [8]:
print("\nRealizando BLASTp LOCAL...")

BLASTP_EXE = "blastp"              
DB_LOCAL = "mi_db_prot"            

with tempfile.TemporaryDirectory() as tmpdir:
    query_path = os.path.join(tmpdir, "query.fasta")
    xml_out = os.path.join(tmpdir, "blast_local.xml")

    with open(query_path, "w") as f:
        f.write(">consulta\n")
        f.write(seq_limpia + "\n")

    blast_cline = NcbiblastpCommandline(
        cmd=BLASTP_EXE,
        query=query_path,
        db=DB_LOCAL,
        outfmt=5,
        out=xml_out,
        evalue=0.001
    )

    stdout, stderr = blast_cline()

    with open(xml_out) as f:
        record_local = NCBIXML.read(f)

    resultados_local = extraer_hits_filtrados(record_local)

    guardar_resultados("resultados_local_proteina.txt", resultados_local)

print("\n--- RESULTADOS LOCALES ---")
print(f"Total hits con E-value < 0.001: {len(resultados_local)}")
print("Guardado en 'resultados_local_proteina.txt'")



Realizando BLASTp LOCAL...

--- RESULTADOS LOCALES ---
Total hits con E-value < 0.001: 0
Guardado en 'resultados_local_proteina.txt'


## 6. Resultados obtenidos
Tanto en la búsqueda online como en la local, los resultados se almacenan en un fichero que contiene exactamente los campos solicitados:

- ID del hit
- Longitud de la proteína
- E-value
- % de identidad

Este formato permite comparar fácilmente las dos estrategias de búsqueda (online vs. local) y evaluar cómo cambian los resultados dependiendo del tamaño y naturaleza de la base de datos utilizada.

## 7. Conclusiones finales

En este ejercicio se ha automatizado una búsqueda BLASTp tanto online como en local para comparar una secuencia de proteína introducida por el usuario frente a diferentes bases de datos proteicas.

En la parte online, se utiliza `NCBIWWW.qblast` para consultar la base de datos `nr` del NCBI, que contiene millones de proteínas anotadas de múltiples organismos. Este enfoque garantiza una cobertura amplia y permite identificar posibles homólogos lejanos, aunque depende de la disponibilidad del servidor y del tiempo de respuesta.

En la parte local, se ejecuta `blastp` sobre una base de datos previamente construida mediante `makeblastdb`. Aunque requiere una preparación inicial (descargar secuencias proteicas y generar la base de datos), una vez configurado resulta más rápido y ofrece un entorno reproducible e independiente de Internet. Además, permite trabajar con bases de datos diseñadas específicamente para una práctica o un proyecto concreto.

En ambos casos se emplea un filtrado por E-value (< 0.001) para seleccionar únicamente los alineamientos estadísticamente significativos. Para cada resultado, se extrae información relevante como:
- El identificador de la proteína coincidente.
- La longitud de la secuencia en la base de datos.
- El E-value del alineamiento.
- El porcentaje de identidad, indicador de la similitud real entre ambas proteínas.

Con esto, como en el ejercicio 1, se demuestra cómo integrar Python, Biopython y BLAST para analizar secuencias proteicas de forma automatizada, interpretar resultados relevantes y comparar la eficiencia y utilidad de búsquedas online frente a búsquedas locales. Este flujo de trabajo es fundamental en bioinformática estructural, anotación funcional y análisis comparativo de proteínas

